# Laboratorio 1 - Estructuras de Datos II


## Imports y Clases


In [4]:
from PIL import Image
import webbrowser
import graphviz as gv
import geopandas as geo
from pprint import pprint
from typing import Any, List, Optional, Tuple
import pandas as pd
import seaborn as sns


def preorder(node: Optional["Node"]) -> None:
    if node is not None:
        print(node.data, end=' ')
        preorder(node.left)
        preorder(node.right)


def inorder(node: Optional["Node"]) -> None:
    if node is not None:
        inorder(node.left)
        print(node.data, end=' ')
        inorder(node.right)


def postorder(node: Optional["Node"]) -> None:
    if node is not None:
        postorder(node.left)
        postorder(node.right)
        print(node.data, end=' ')


class Node:

    def __init__(self, data: Any, lvl=0) -> None:
        self.data = data
        self.lvl = lvl
        self.left: Optional["Node"] = None
        self.right: Optional["Node"] = None

    def __repr__(self) -> str:
        left = self.left.data if self.left is not None else None
        right = self.right.data if self.right is not None else None
        return f'{self.data} {left} {right}'


class AVL:

    def __init__(self) -> None:
        self.root: Optional["Node"] = None

    def aunt(self, node1, node2: Optional["Node"], node3: Optional["Node"], elem):
        if node1 is not None:
            if elem == node1.data["price"]/node1.data["surface_total"]:
                return node3
            elif elem < node1.data["price"]/node1.data["surface_total"]:
                if node2 is not None:
                    if node1 == node2.left:
                        return self.aunt(node1.left, node1, node2.right, elem)
                    else:
                        return self.aunt(node1.left, node1, node2.left, elem)
                else:
                    return self.aunt(node1.left, node1, None, elem)
            else:
                if node2 is not None:
                    if node1 == node2.left:
                        return self.aunt(node1.right, node1, node2.right, elem)
                    else:
                        return self.aunt(node1.right, node1, node2.left, elem)
                else:
                    return self.aunt(node1.right, node1, None, elem)
        return None

    def search(self, elem: Any) -> Tuple[Optional["Node"], Optional["Node"]]:
        p, pad = self.root, None
        while p is not None:
            if elem == p.data:
                return p, pad
            else:
                pad = p
                if elem < p.data:
                    p = p.left
                else:
                    p = p.right

        return p, pad

    def search_metric1(self, elem: float) -> Tuple[Optional["Node"], Optional["Node"]]:
        p, pad = self.root, None
        while p is not None:
            if elem == p.data["price"]/p.data["surface_total"]:
                return p, pad
            else:
                pad = p
                if elem < p.data["price"]/p.data["surface_total"]:
                    p = p.left
                else:
                    p = p.right

        return p, pad

    def add_edges(self, graph, node):
        if node.left:
            graph.edge(str(node.data), str(node.left.data))
            self.add_edges(graph, node.left)
        if node.right:
            graph.edge(str(node.data), str(node.right.data))
            self.add_edges(graph, node.right)

    def visualize_tree_pdf(self):
        graph = gv.Digraph('G', format='pdf')
        self.add_edges(graph, self.root)
        graph.render('tree_pdf')

        pdf_file_path = 'tree_pdf.pdf'
        webbrowser.open(pdf_file_path)

    def balance_node(self, node: "Node") -> "Node":
        # Calcula la altura de los subárboles izquierdo y derecho
        left_height = self.height(node.left)
        right_height = self.height(node.right)

        # Calcula el factor de equilibrio del nodo actual
        balance_factor = left_height - right_height

        # Si el factor de equilibrio es mayor que 1, el subárbol izquierdo es más alto
        if balance_factor > 1:
            # Verifica si se requiere una rotación simple o doble
            if self.height(node.left.left) >= self.height(node.left.right):
                # Rotación simple derecha
                return self.right_rotate(node)
            else:
                # Rotación doble izquierda-derecha
                node.left = self.left_rotate(node.left)
                return self.right_rotate(node)

        # Si el factor de equilibrio es menor que -1, el subárbol derecho es más alto
        if balance_factor < -1:
            # Verifica si se requiere una rotación simple o doble
            if self.height(node.right.right) >= self.height(node.right.left):
                # Rotación simple izquierda
                return self.left_rotate(node)
            else:
                # Rotación doble derecha-izquierda
                node.right = self.right_rotate(node.right)
                return self.left_rotate(node)

        # Si el factor de equilibrio está en el rango [-1, 1], el nodo está equilibrado
        return node

    def insert(self, elem: Any) -> bool:
        to_insert = Node(elem)
        if self.root is None:
            self.root = to_insert
            return True
        else:
            p, pad = self.search_metric1(elem["price"]/elem["surface_total"])
            if p is None:
                if elem["price"]/elem["surface_total"] < pad.data["price"]/pad.data["surface_total"]:
                    pad.left = to_insert
                    pad.left.lvl = pad.lvl + 1
                else:
                    pad.right = to_insert
                    pad.right.lvl = pad.lvl + 1
                # Después de la inserción, balancea el árbol
                self.root = self.balance_node(self.root)
                return True
            return False

    def delete(self, elem: Any, mode: bool = True) -> bool:
        p, pad = self.search_metric1(elem["price"]/elem["surface_total"])
        if p is not None:
            if p.left is None and p.right is None:
                if p == pad.left:
                    pad.left = None
                else:
                    pad.right = None
                del p
            elif p.left is None and p.right is not None:
                if p == pad.left:
                    pad.left = p.right
                else:
                    pad.right = p.right
                del p
            elif p.left is not None and p.right is None:
                if p == pad.left:
                    pad.left = p.left
                else:
                    pad.right = p.left
                del p
            else:
                if mode:
                    pred, pad_pred = self.__pred(p)
                    p.data = pred.data
                    if pred.left is not None:
                        pad_pred.right = pred.left
                    else:
                        pad_pred.right = None
                    del pred
                else:
                    sus, pad_sus = self.__sus(p)
                    p.data = sus.data
                    if sus.right is not None:
                        pad_sus.left = sus.right
                    else:
                        pad_sus.left = None
                    del sus
            # Después de la eliminación, balancea el árbol
            self.root = self.balance_node(self.root)
            return True
        return False

    def __pred(self, node: "Node") -> Tuple["Node", "Node"]:
        p, pad = node.left, node
        while p.right is not None:
            p, pad = p.right, p

        return p, pad

    def __sus(self, node: "Node") -> Tuple["Node", "Node"]:
        p, pad = node.right, node
        while p.left is not None:
            p, pad = p.left, p

        return p, pad

    def __inorder(self, node: Optional["Node"]) -> None:
        if node is not None:
            self.__inorder(node.left)
            print(node)
            self.__inorder(node.right)

    def inorder(self) -> None:
        self.__inorder(self.root)

    def left_rotate(self, node: "Node") -> "Node":
        new_root = node.right
        new_root.lvl = node.lvl
        node.right = new_root.left
        node.right.lvl = node.lvl + 1
        new_root.left = node
        new_root.left.lvl = node.lvl + 1
        return new_root

    def right_rotate(self, node: "Node") -> "Node":
        new_root = node.left
        new_root.lvl = node.lvl
        node.left = new_root.right
        node.left.lvl = node.lvl + 1
        new_root.right = node
        new_root.right.lvl = node.lvl + 1
        return new_root

    def height(self, node):
        if node is None:
            return 0
        else:
            return max(self.height(node.left), self.height(node.right)) + 1

## CSV Visualización


In [8]:
df = pd.read_csv("co_properties_final.csv")
tree = AVL()

for i in range(len(df)):
    fila = df.iloc[i]
    tree.insert(fila)


None


In [3]:
tree.visualize_tree_pdf()


department                                          Valle del Cauca
city                                                           Cali
property_type                                                  Casa
latitude                                                      3.368
longitude                                                   -76.531
surface_total                                                 420.0
surface_covered                                               420.0
bedrooms                                                        5.0
bathrooms                                                       5.0
operation_type                                                Venta
price                                                   950000000.0
Name, port  1, dtype unrecognized
department                                      Cundinamarca
city                                                    Sopó
property_type                                           Casa
latitude                           